In [ ]:
import folium
lat,lon=[35.0007407156917, 135.7408483539235] # 京都近辺の緯度経度
def create_map(zoom_level,gj_files,tj_files):  # 地図を描く関数
    fmap = folium.Map(location=[lat,lon],
        zoom_start=zoom_level,attr='OpenStreetMap & 国土地理院地図')
    if gj_files:
        for gj in gj_files:
            style={'color':'#FF3333E0','weight':1}
            folium.GeoJson(open(gj['file']).read(),name=gj['name'],
                style_function=lambda x:style).add_to(fmap)
    if tj_files:
        for tj in tj_files:
            topojson_path = './data/day_240526_'+tj['file']
            folium.TopoJson(
                data=open(topojson_path, encoding='utf-8'),
                object_path='objects.town',name=tj['name']
            ).add_to(fmap)    
    folium.LayerControl().add_to(fmap)
    return fmap
gj_files=[
{'name':'平安京跡データベース/立命館大学アート・リサーチセンター,武内 樹治',
'file':'data/heiankyo_residence.geojson'}]
create_map(13,gj_files,None)

In [ ]:
import ephem
kyoto=ephem.Observer() # 場所を設定
kyoto.lat=str(lat);kyoto.lon=str(lon) # 弧度はstrにする
sun = ephem.Sun()      # 太陽情報を計算
kyoto.date = ephem.Date('2024/9/23 18:0:0.0')
sunrise = kyoto.next_rising(sun)
print("春分・秋分の日の出時間:", ephem.localtime(sunrise))
kyoto.date=sunrise;sun.compute(kyoto) # 太陽の方向を計算
print('方位角（北基準で東方向・時計回り、単位は度）：',sun.az)

In [ ]:
kyoto.date = ephem.Date('2024/06/20 18:0:0.0')
print("夏至近くの日の出:", ephem.localtime(kyoto.next_rising(sun)))
kyoto.date = ephem.Date('2024/12/20 18:0:0.0')
print("冬至近くの日の出:", ephem.localtime(kyoto.next_rising(sun)))

In [ ]:
# TopoJsonデータ
topojson_files=[
    {'name':'上京区','file':'r2ka26102.topojson'},
    {'name':'左京区','file':'r2ka26103.topojson'},
    {'name':'右京区','file':'r2ka26108.topojson'},
    {'name':'東山区','file':'r2ka26105.topojson'},
    {'name':'西京区','file':'r2ka26111.topojson'},
    {'name':'中京区','file':'r2ka26104.topojson'},
    {'name':'下京区','file':'r2ka26106.topojson'},
    {'name':'伏見区','file':'r2ka26109.topojson'},
    {'name':'山科区','file':'r2ka26110.topojson'},
    {'name':'南区','file':'r2ka26107.topojson'},
    {'name':'北区','file':'r2ka26101.topojson'}]
create_map(13,None,topojson_files)

In [ ]:
import networkx as nx
from IPython.display import display,SVG
def draw(graph):
    svg=nx.nx_agraph.to_agraph(graph).draw(prog='dot',format='svg')
    display(SVG(svg))
g = nx.MultiGraph()
g.add_nodes_from(["浪華A","浪華B","浪華C","浪華D","浪華E","浪華F","浪華G"])
g.add_edges_from([("浪華A","浪華B"),("浪華A","浪華B"),("浪華A","浪華D"),
("浪華A","浪華D"),("浪華B","浪華C"),("浪華B","浪華F"),("浪華B","浪華B"),
("浪華C","浪華G"),("浪華D","浪華E"),("浪華D","浪華F"),("浪華D","浪華F"),
("浪華D","浪華F"),("浪華E","浪華F"),("浪華F","浪華G"),("浪華F","浪華G"),
("浪華F","浪華G")])
print("島：",g.number_of_nodes());print("橋：",g.number_of_edges());draw(g)
if nx.is_eulerian(g):
    print('オイラー閉路があります。')
elif nx.is_semieulerian(g):
    print('オイラー路があります（閉路はありません）。')
else:
    print('オイラー路も閉路もありません。')
# 一筆書き経路を描く
for (u,v) in list(nx.eulerian_circuit(g)):
    print(u,'から',v,'に渡る。')

In [ ]:
# グラフを取得
nodes, edges = ox.graph_to_gdfs(graph)
# グラフのノードとエッジを追加
nxgraph = nx.Graph()
for idx, data in edges.iterrows():
    u, v, key = idx
    nxgraph.add_edge(u, v)
    
    # 橋に絞る
bridge_edges = edges[edges['bridge'] == 'yes']
# グラフのノードとエッジを追加
bridge_nxgraph = nx.Graph()
for idx, data in bridge_edges.iterrows():
    u, v, key = idx
    bridge_nxgraph.add_edge(u, v)
# 地図表示する
fmap = folium.Map(location=aiai, zoom_start=14)

# グラフを地図に追加
for u, v in nxgraph.edges():
    point_u = (graph.nodes[u]['y'],graph.nodes[u]['x'])
    point_v = (graph.nodes[v]['y'],graph.nodes[v]['x'])
    folium.PolyLine(locations=[point_u, point_v], color='#0000FF20').add_to(fmap)
# グラフを地図に追加
for u, v in bridge_nxgraph.edges():
    point_u = (graph.nodes[u]['y'],graph.nodes[u]['x'])
    point_v = (graph.nodes[v]['y'],graph.nodes[v]['x'])
    folium.PolyLine(locations=[point_u, point_v], color='#FF0000FF').add_to(fmap)

folium.Marker(location=kengyu, popup="牽牛石",
              icon=folium.Icon(color='blue')).add_to(fmap)
folium.Marker(location=hatamono, popup="機物神社",
              icon=folium.Icon(color='red')).add_to(fmap)
folium.Marker(location=aiai, popup="逢合橋",
              icon=folium.Icon(color='green')).add_to(fmap)
folium.LayerControl().add_to(fmap)
folium.PolyLine(route_coords, color="blue", weight=2.5, opacity=1).add_to(fmap)
fmap